Extract the data from http://www.football-data.co.uk/, Transform them and Load them in a Database.

In [1]:
# Import library
import json
import sqlite3
import requests
import io
import pandas as pd # data processing, CSV file I/O
from IPython.display import display # Manage multiple output per cell

In [2]:
# DB Sqlite connection
db = "/Users/thibaultclement/Project/ligue1-predict/src/notebook/data/db/soccer_predict.sqlite"
conn = sqlite3.connect(db)
cur = conn.cursor()

In [3]:
# Clean Database
cur.execute('DROP TABLE IF EXISTS pre_matchs_v4')
conn.commit()

In [4]:
# Get all data for pre match on away team
def homeData( date, team, div, nb_matches, nb_matches_string ):
    # Dataframe to return with all info
    dic = {}
    # Home team query
    #TODO Recuperer aussi combien de buts ils se sont pris dans la tronche et tout et tout
    queryHome = '''
            SELECT Date, FTHG, FTR, HTHG, HTR, HS, HST, HF, HC, HY, HR, FTAG, HTAG, `AS`, AST, AF, AC, AY, AR
            FROM macths_raw
            WHERE Date < ? AND HomeTeam = ? AND Div = ? ORDER BY Date DESC LIMIT ?'''
    # Get the previous home game of the Home Team
    df_home = pd.read_sql(queryHome, conn, params=[date, team, div, nb_matches])
    # Hot-encode Category Full Time Result and Half Time Result + reverse order
    df_home = pd.get_dummies(df_home, columns=['FTR', 'HTR']).iloc[::-1]
    # Calculate the mean of all columns
    #display(df_home.head())
    dic['H_MEANS_'+nb_matches_string+'_FTHG'] = round(df_home.FTHG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_home.columns else round(df_home.FTR_H.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_home.columns else round(df_home.FTR_D.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_home.columns else round(df_home.FTR_A.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTHG'] = round(df_home.HTHG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_home.columns else round(df_home.HTR_H.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_home.columns else round(df_home.HTR_D.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_home.columns else round(df_home.HTR_A.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HS'] = round(df_home.HS.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HST'] = round(df_home.HST.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HF'] = round( df_home.HF.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HC'] = round(df_home.HC.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HY'] = round(df_home.HY.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HR'] = round(df_home.HR.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTAG'] = round(df_home.FTAG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTAG'] = round(df_home.HTAG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AS'] = round(df_home.AS.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AST'] = round(df_home.AST.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AF'] = round(df_home.AF.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AC'] = round(df_home.AC.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AY'] = round(df_home.AY.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AR'] = round(df_home.AR.mean(), 2)
    dic['H_STD_'+nb_matches_string+'_FTHG'] = round((df_home.FTHG.shift(1) - df_home.FTHG).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_home.columns else round((df_home.FTR_H.shift(1) - df_home.FTR_H).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_home.columns else round((df_home.FTR_D.shift(1) - df_home.FTR_D).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_home.columns else round((df_home.FTR_A.shift(1) - df_home.FTR_A).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HTHG'] = round((df_home.HTHG.shift(1) - df_home.HTHG).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_home.columns else round((df_home.HTR_H.shift(1) - df_home.HTR_H).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_home.columns else round((df_home.HTR_D.shift(1) - df_home.HTR_D).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_home.columns else round((df_home.HTR_A.shift(1) - df_home.HTR_A).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HS'] = round((df_home.HS.shift(1) - df_home.HS).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HST'] = round((df_home.HST.shift(1) - df_home.HST).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HF'] = round((df_home.HF.shift(1) - df_home.HF).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HC'] = round((df_home.HC.shift(1) - df_home.HC).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HY'] = round((df_home.HY.shift(1) - df_home.HY).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HR'] = round((df_home.HR.shift(1) - df_home.HR).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_FTAG'] = round((df_home.FTAG.shift(1) - df_home.FTAG).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_HTAG'] = round((df_home.HTAG.shift(1) - df_home.HTAG).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_AS'] = round((df_home.AS.shift(1) - df_home.AS).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_AST'] = round((df_home.AST.shift(1) - df_home.AST).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_AF'] = round((df_home.AF.shift(1) - df_home.AF).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_AC'] = round((df_home.AC.shift(1) - df_home.AC).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_AY'] = round((df_home.AY.shift(1) - df_home.AY).mean(), 3)
    dic['H_STD_'+nb_matches_string+'_AR'] = round((df_home.AR.shift(1) - df_home.AR).mean(), 3)
    return dic

In [5]:
# Get all data for pre match on away team
def awayData( date, team, div, nb_matches, nb_matches_string ):
    # Dataframe to return with all info
    dic = {}
    # away team query
    #TODO Recuperer aussi combien de buts ils se sont pris dans la tronche et tout et tout
    queryAway = '''
            SELECT Date, FTAG, FTR, HTAG, HTR, `AS`, AST, AF, AC, AY, AR, FTHG, HTHG, HS, HST, HF, HC, HY, HR
            FROM macths_raw
            WHERE Date < ? AND AwayTeam = ? AND Div = ? ORDER BY Date DESC LIMIT ?'''
    # Get the previous away game of the away Team
    df_away = pd.read_sql(queryAway, conn, params=[date, team, div, nb_matches])
    # Hot-encode Category Full Time Result and Half Time Result
    df_away = pd.get_dummies(df_away, columns=['FTR', 'HTR']).iloc[::-1]
    # Calculate the mean of all columns
    #display(df_away.head())
    dic['A_MEANS_'+nb_matches_string+'_FTAG'] = round(df_away.FTAG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_away.columns else round(df_away.FTR_H.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_away.columns else round(df_away.FTR_D.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_away.columns else round(df_away.FTR_A.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTAG'] = round(df_away.HTAG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_away.columns else round(df_away.HTR_H.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_away.columns else round(df_away.HTR_D.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_away.columns else round(df_away.HTR_A.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AS'] = round(df_away.AS.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AST'] = round(df_away.AST.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AF'] = round(df_away.AF.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AC'] = round(df_away.AC.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AY'] = round(df_away.AY.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AR'] = round(df_away.AR.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTHG'] = round(df_away.FTHG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTHG'] = round(df_away.HTHG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HS'] = round(df_away.HS.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HST'] = round(df_away.HST.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HF'] = round( df_away.HF.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HC'] = round(df_away.HC.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HY'] = round(df_away.HY.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HR'] = round(df_away.HR.mean(), 2)
    dic['A_STD_'+nb_matches_string+'_FTHG'] = round((df_away.FTHG.shift(1) - df_away.FTHG).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_away.columns else round((df_away.FTR_H.shift(1) - df_away.FTR_H).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_away.columns else round((df_away.FTR_D.shift(1) - df_away.FTR_D).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_away.columns else round((df_away.FTR_A.shift(1) - df_away.FTR_A).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HTHG'] = round((df_away.HTHG.shift(1) - df_away.HTHG).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_away.columns else round((df_away.HTR_H.shift(1) - df_away.HTR_H).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_away.columns else round((df_away.HTR_D.shift(1) - df_away.HTR_D).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_away.columns else round((df_away.HTR_A.shift(1) - df_away.HTR_A).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HS'] = round((df_away.HS.shift(1) - df_away.HS).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HST'] = round((df_away.HST.shift(1) - df_away.HST).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HF'] = round((df_away.HF.shift(1) - df_away.HF).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HC'] = round((df_away.HC.shift(1) - df_away.HC).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HY'] = round((df_away.HY.shift(1) - df_away.HY).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HR'] = round((df_away.HR.shift(1) - df_away.HR).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_FTAG'] = round((df_away.FTAG.shift(1) - df_away.FTAG).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_HTAG'] = round((df_away.HTAG.shift(1) - df_away.HTAG).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_AS'] = round((df_away.AS.shift(1) - df_away.AS).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_AST'] = round((df_away.AST.shift(1) - df_away.AST).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_AF'] = round((df_away.AF.shift(1) - df_away.AF).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_AC'] = round((df_away.AC.shift(1) - df_away.AC).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_AY'] = round((df_away.AY.shift(1) - df_away.AY).mean(), 3)
    dic['A_STD_'+nb_matches_string+'_AR'] = round((df_away.AR.shift(1) - df_away.AR).mean(), 3)
    return dic

In [6]:
# Get all data for pre match on away team
def homeAndAwayData( date, team, div, nb_matches, nb_matches_string ):
    # Dataframe to return with all info
    dic = {}
    # away team query
    queryAway = '''
            SELECT Date, AwayTeam, HomeTeam, FTAG, FTR, HTAG, HTR, `AS`, AST, AF, AC, AY, AR, FTHG, HTHG, HS, HST, HF, HC, HY, HR
            FROM macths_raw
            WHERE Date < ? AND (AwayTeam = ? OR HomeTeam = ?) AND Div = ? ORDER BY Date DESC LIMIT ?'''
    # Get the previous game of the Team
    df = pd.read_sql(queryAway, conn, params=[date, team, team, div, nb_matches])
    # Hot-encode Category Full Time Result and Half Time Result
    df = pd.get_dummies(df, columns=['FTR', 'HTR'])
    # Split based on when the team play at home or away
    df_away = df[df['AwayTeam'] == team]
    away_nb = len(df_away.index)
    df_home = df[df['HomeTeam'] == team]
    home_nb = len(df_home.index)
    game_nb = 1.0 if (away_nb + home_nb) == 0 else float(away_nb + home_nb) # Avoid division by zero
    # Calculate the mean of all columns
    #display(df_away.head())
    dic['HandA_'+nb_matches_string+'_FTAG'] = round((df_away.FTAG.sum()+df_home.FTHG.sum())/game_nb, 2)
    away_FTR_H = 0 if 'FTR_H'not in df_away.columns else df_away.FTR_H.sum()
    home_FTR_H = 0 if 'FTR_H'not in df_home.columns else df_home.FTR_H.sum()
    dic['HandA_'+nb_matches_string+'_FTR_H'] = round((away_FTR_H+home_FTR_H)/game_nb, 2)
    away_FTR_D = 0 if 'FTR_D'not in df_away.columns else df_away.FTR_D.sum()
    home_FTR_D = 0 if 'FTR_D'not in df_home.columns else df_home.FTR_D.sum()
    dic['HandA_'+nb_matches_string+'_FTR_D'] = round((away_FTR_D+home_FTR_D)/game_nb, 2)
    away_FTR_A = 0 if 'FTR_A'not in df_away.columns else df_away.FTR_A.sum()
    home_FTR_A = 0 if 'FTR_A'not in df_home.columns else df_home.FTR_A.sum()
    dic['HandA_'+nb_matches_string+'_FTR_A'] = round((away_FTR_A+home_FTR_A)/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HTAG'] = round((df_away.HTAG.sum()+df_home.HTHG.sum())/game_nb, 2)
    away_HTR_H = 0 if 'HTR_H'not in df_away.columns else df_away.HTR_H.sum()
    home_HTR_H = 0 if 'HTR_H'not in df_home.columns else df_home.HTR_H.sum()
    dic['HandA_'+nb_matches_string+'_HTR_H'] = round((away_HTR_H+home_HTR_H)/game_nb, 2)
    away_HTR_D = 0 if 'HTR_D'not in df_away.columns else df_away.HTR_D.sum()
    home_HTR_D = 0 if 'HTR_D'not in df_home.columns else df_home.HTR_D.sum()
    dic['HandA_'+nb_matches_string+'_HTR_D'] = round((away_HTR_D+home_HTR_D)/game_nb, 2)
    away_HTR_A = 0 if 'HTR_A'not in df_away.columns else df_away.HTR_A.sum()
    home_HTR_A = 0 if 'HTR_A'not in df_home.columns else df_home.HTR_A.sum()
    dic['HandA_'+nb_matches_string+'_HTR_A'] = round((away_HTR_A+home_HTR_A)/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_AS'] = round((df_away.AS.sum()+df_home.HS.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_AST'] = round((df_away.AST.sum()+df_home.HST.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_AF'] = round((df_away.AF.sum()+df_home.HF.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_AC'] = round((df_away.AC.sum()+df_home.HC.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_AY'] = round((df_away.AY.sum()+df_home.HY.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_AR'] = round((df_away.AR.sum()+df_home.HR.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_FTHG'] = round((df_away.FTHG.sum()+df_home.FTAG.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HTHG'] = round((df_away.HTHG.sum()+df_home.HTAG.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HS'] = round((df_away.HS.sum()+df_home.AS.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HST'] = round((df_away.HST.sum()+df_home.AST.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HF'] = round((df_away.HF.sum()+df_home.AF.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HC'] = round((df_away.HC.sum()+df_home.AC.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HY'] = round((df_away.HY.sum()+df_home.AY.sum())/game_nb, 2)
    dic['HandA_'+nb_matches_string+'_HR'] = round((df_away.HR.sum()+df_home.AR.sum())/game_nb, 2)
    return dic

In [7]:
# Get all data for pre match on away team
def awayAndHomeData( date, team, div, nb_matches, nb_matches_string ):
    # Dataframe to return with all info
    dic = {}
    # away team query
    queryAway = '''
            SELECT Date, AwayTeam, HomeTeam, FTAG, FTR, HTAG, HTR, `AS`, AST, AF, AC, AY, AR, FTHG, HTHG, HS, HST, HF, HC, HY, HR
            FROM macths_raw
            WHERE Date < ? AND (AwayTeam = ? OR HomeTeam = ?) AND Div = ? ORDER BY Date DESC LIMIT ?'''
    # Get the previous game of the Team
    df = pd.read_sql(queryAway, conn, params=[date, team, team, div, nb_matches])
    # Hot-encode Category Full Time Result and Half Time Result
    df = pd.get_dummies(df, columns=['FTR', 'HTR'])
    # Split based on when the team play at home or away
    df_away = df[df['AwayTeam'] == team]
    away_nb = len(df_away.index)
    df_home = df[df['HomeTeam'] == team]
    home_nb = len(df_home.index)
    game_nb = 1.0 if (away_nb + home_nb) == 0 else float(away_nb + home_nb) # Avoid division by zero
    # Calculate the mean of all columns
    #display(df_away.head())
    dic['AandH_'+nb_matches_string+'_FTAG'] = round((df_away.FTAG.sum()+df_home.FTHG.sum())/game_nb, 2)
    away_FTR_H = 0 if 'FTR_H'not in df_away.columns else df_away.FTR_H.sum()
    home_FTR_H = 0 if 'FTR_H'not in df_home.columns else df_home.FTR_H.sum()
    dic['AandH_'+nb_matches_string+'_FTR_H'] = round((away_FTR_H+home_FTR_H)/game_nb, 2)
    away_FTR_D = 0 if 'FTR_D'not in df_away.columns else df_away.FTR_D.sum()
    home_FTR_D = 0 if 'FTR_D'not in df_home.columns else df_home.FTR_D.sum()
    dic['AandH_'+nb_matches_string+'_FTR_D'] = round((away_FTR_D+home_FTR_D)/game_nb, 2)
    away_FTR_A = 0 if 'FTR_A'not in df_away.columns else df_away.FTR_A.sum()
    home_FTR_A = 0 if 'FTR_A'not in df_home.columns else df_home.FTR_A.sum()
    dic['AandH_'+nb_matches_string+'_FTR_A'] = round((away_FTR_A+home_FTR_A)/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HTAG'] = round((df_away.HTAG.sum()+df_home.HTHG.sum())/game_nb, 2)
    away_HTR_H = 0 if 'HTR_H'not in df_away.columns else df_away.HTR_H.sum()
    home_HTR_H = 0 if 'HTR_H'not in df_home.columns else df_home.HTR_H.sum()
    dic['AandH_'+nb_matches_string+'_HTR_H'] = round((away_HTR_H+home_HTR_H)/game_nb, 2)
    away_HTR_D = 0 if 'HTR_D'not in df_away.columns else df_away.HTR_D.sum()
    home_HTR_D = 0 if 'HTR_D'not in df_home.columns else df_home.HTR_D.sum()
    dic['AandH_'+nb_matches_string+'_HTR_D'] = round((away_HTR_D+home_HTR_D)/game_nb, 2)
    away_HTR_A = 0 if 'HTR_A'not in df_away.columns else df_away.HTR_A.sum()
    home_HTR_A = 0 if 'HTR_A'not in df_home.columns else df_home.HTR_A.sum()
    dic['AandH_'+nb_matches_string+'_HTR_A'] = round((away_HTR_A+home_HTR_A)/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_AS'] = round((df_away.AS.sum()+df_home.HS.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_AST'] = round((df_away.AST.sum()+df_home.HST.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_AF'] = round((df_away.AF.sum()+df_home.HF.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_AC'] = round((df_away.AC.sum()+df_home.HC.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_AY'] = round((df_away.AY.sum()+df_home.HY.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_AR'] = round((df_away.AR.sum()+df_home.HR.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_FTHG'] = round((df_away.FTHG.sum()+df_home.FTAG.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HTHG'] = round((df_away.HTHG.sum()+df_home.HTAG.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HS'] = round((df_away.HS.sum()+df_home.AS.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HST'] = round((df_away.HST.sum()+df_home.AST.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HF'] = round((df_away.HF.sum()+df_home.AF.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HC'] = round((df_away.HC.sum()+df_home.AC.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HY'] = round((df_away.HY.sum()+df_home.AY.sum())/game_nb, 2)
    dic['AandH_'+nb_matches_string+'_HR'] = round((df_away.HR.sum()+df_home.AR.sum())/game_nb, 2)
    return dic

In [8]:
# Compute all pre matchs and insert them in pre_match table
df_macths_raw = pd.read_sql_query("SELECT Div, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR, HS, `AS`, HST, AST, HF, AF, HC, AC, HY, AY, HR, AR, PSH, PSD, PSA, BbAvH, BbAvD, BbAvA FROM macths_raw ORDER BY Date ASC;", conn)
#df_macths_raw = pd.read_sql_query("SELECT Div, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR, HS, `AS`, HST, AST, HF, AF, HC, AC, HY, AY, HR, AR, PSH, PSD, PSA, BbAvH, BbAvD, BbAvA FROM macths_raw ORDER BY Date DESC LIMIT 1;", conn)
for index, row in df_macths_raw.iterrows():
    #display(row)
    
    # HOME TEAM
    # Get the 5 previous home game of the Home Team
    dic_home_five = homeData(row.Date, row.HomeTeam, row.Div, 5, 'FIVE')
    #display(df_home_five)
    # Get the 3 previous home game of the Home Team
    dic_home_three = homeData(row.Date, row.HomeTeam, row.Div, 3, 'THREE')
    #display(df_home_three)
    # Get the 5 previous game of the Home Team
    dic_homeAndAway_seven = homeAndAwayData(row.Date, row.HomeTeam, row.Div, 7, 'SEVEN')
    
    # AWAY TEAM
    # Get the 5 previous away game of the Away Team
    dic_away_five = awayData(row.Date, row.AwayTeam, row.Div, 5, 'FIVE')
    #display(df_away_five)
    # Get the 3 previous away game of the Away Team
    dic_away_three = awayData(row.Date, row.AwayTeam, row.Div, 3, 'THREE')
    #display(df_away_three)
    # Get the 5 previous game of the Away Team
    dic_awayAndHome_seven = awayAndHomeData(row.Date, row.AwayTeam, row.Div, 7, 'SEVEN')
    
    # Merge different dataset together
    #dic_all = dic_home_five.copy().update(dic_home_three)
    dic_all = dict(dic_home_five.items() + dic_home_three.items() + dic_homeAndAway_seven.items() + dic_away_five.items() + dic_away_three.items() + dic_awayAndHome_seven.items())
    # Add division and date
    dic_all['INFO_Div'] = row['Div']
    dic_all['INFO_Date'] = row['Date']
    # Add info on result
    dic_all['INFO_HomeTeam'] = row['HomeTeam']
    dic_all['INFO_AwayTeam'] = row['AwayTeam']
    dic_all['INFO_FTR'] = row['FTR']
    dic_all['INFO_HTR'] = row['HTR']
    dic_all['INFO_FTHG'] = row['FTHG']
    dic_all['INFO_FTAG'] = row['FTAG']
    # Add all Bet
    dic_all['INFO_PSH'] = row['PSH']
    dic_all['INFO_PSD'] = row['PSD']
    dic_all['INFO_PSA'] = row['PSA']
    dic_all['INFO_BbAvH'] = row['BbAvH']
    dic_all['INFO_BbAvD'] = row['BbAvD']
    dic_all['INFO_BbAvA'] = row['BbAvA']
    df_pre_matches = pd.DataFrame.from_dict([dic_all], orient='columns')
    df_pre_matches.to_sql("pre_matchs_v4", conn, if_exists="append")

df_macths_raw.shape

(37907, 28)